In [1]:
!wget https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testb

--2022-03-03 15:18:33--  https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 748096 (731K) [text/plain]
Saving to: ‘eng.testb.1’

eng.testb.1         100%[===================>] 730.56K  --.-KB/s    in 0.05s   

2022-03-03 15:18:33 (15.3 MB/s) - ‘eng.testb.1’ saved [748096/748096]



In [2]:
!pip install datasets

In [3]:
import unicodedata
import spacy
import numpy as np
from tqdm import tqdm, trange

In [5]:
!pip install --upgrade spacy

     |████████████████████████████████| 6.0 MB 5.3 MB/s 
     |████████████████████████████████| 628 kB 41.0 MB/s 
     |████████████████████████████████| 451 kB 43.8 MB/s 
     |████████████████████████████████| 181 kB 37.2 MB/s 
     |████████████████████████████████| 10.1 MB 34.5 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [21]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 777.4 MB 6.8 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [23]:
nlp = spacy.load("en_core_web_lg")

In [5]:
!unzip nlp-project-dev-data-articles.zip

Archive:  nlp-project-dev-data-articles.zip
   creating: nlp-project-dev-data-articles/
  inflating: __MACOSX/._nlp-project-dev-data-articles  
  inflating: nlp-project-dev-data-articles/set3-languages.txt  
  inflating: __MACOSX/nlp-project-dev-data-articles/._set3-languages.txt  
  inflating: nlp-project-dev-data-articles/.DS_Store  
  inflating: __MACOSX/nlp-project-dev-data-articles/._.DS_Store  
   creating: nlp-project-dev-data-articles/set2/
  inflating: __MACOSX/nlp-project-dev-data-articles/._set2  
   creating: nlp-project-dev-data-articles/set4/
  inflating: __MACOSX/nlp-project-dev-data-articles/._set4  
   creating: nlp-project-dev-data-articles/set3/
  inflating: __MACOSX/nlp-project-dev-data-articles/._set3  
  inflating: nlp-project-dev-data-articles/set1-football.txt  
  inflating: __MACOSX/nlp-project-dev-data-articles/._set1-football.txt  
   creating: nlp-project-dev-data-articles/set1/
  inflating: __MACOSX/nlp-project-dev-data-articles/._set1  
  inflating: nlp-pr

In [ ]:
# !pip install seqeval git+https://github.com/huggingface/transformers.git
from transformers import LukeTokenizer, LukeForEntitySpanClassification

In [ ]:
# https://colab.research.google.com/github/studio-ousia/luke/blob/master/notebooks/huggingface_conll_2003.ipynb

def load_documents(dataset_file):
    documents = []
    words = []
    labels = []
    sentence_boundaries = []
    with open(dataset_file) as f:
        for line in f:
            line = line.rstrip()
            if line.startswith("-DOCSTART"):
                if words:
                    documents.append(dict(
                        words=words,
                        labels=labels,
                        sentence_boundaries=sentence_boundaries
                    ))
                    words = []
                    labels = []
                    sentence_boundaries = []
                continue

            if not line:
                if not sentence_boundaries or len(words) != sentence_boundaries[-1]:
                    sentence_boundaries.append(len(words))
            else:
                items = line.split(" ")
                words.append(items[0])
                labels.append(items[-1])

    if words:
        documents.append(dict(
            words=words,
            labels=labels,
            sentence_boundaries=sentence_boundaries
        ))
        
    return documents

def load_examples(documents):
    examples = []
    max_token_length = 510
    max_mention_length = 30

    for document in tqdm(documents):
        words = document["words"]
        subword_lengths = [len(tokenizer.tokenize(w)) for w in words]
        total_subword_length = sum(subword_lengths)
        sentence_boundaries = document["sentence_boundaries"]

        for i in range(len(sentence_boundaries) - 1):
            sentence_start, sentence_end = sentence_boundaries[i:i+2]
            if total_subword_length <= max_token_length:
                # if the total sequence length of the document is shorter than the
                # maximum token length, we simply use all words to build the sequence
                context_start = 0
                context_end = len(words)
            else:
                # if the total sequence length is longer than the maximum length, we add
                # the surrounding words of the target sentence　to the sequence until it
                # reaches the maximum length
                context_start = sentence_start
                context_end = sentence_end
                cur_length = sum(subword_lengths[context_start:context_end])
                while True:
                    if context_start > 0:
                        if cur_length + subword_lengths[context_start - 1] <= max_token_length:
                            cur_length += subword_lengths[context_start - 1]
                            context_start -= 1
                        else:
                            break
                    if context_end < len(words):
                        if cur_length + subword_lengths[context_end] <= max_token_length:
                            cur_length += subword_lengths[context_end]
                            context_end += 1
                        else:
                            break

            text = ""
            for word in words[context_start:sentence_start]:
                if word[0] == "'" or (len(word) == 1 and is_punctuation(word)):
                    text = text.rstrip()
                text += word
                text += " "

            sentence_words = words[sentence_start:sentence_end]
            sentence_subword_lengths = subword_lengths[sentence_start:sentence_end]

            word_start_char_positions = []
            word_end_char_positions = []
            for word in sentence_words:
                if word[0] == "'" or (len(word) == 1 and is_punctuation(word)):
                    text = text.rstrip()
                word_start_char_positions.append(len(text))
                text += word
                word_end_char_positions.append(len(text))
                text += " "

            for word in words[sentence_end:context_end]:
                if word[0] == "'" or (len(word) == 1 and is_punctuation(word)):
                    text = text.rstrip()
                text += word
                text += " "
            text = text.rstrip()

            entity_spans = []
            original_word_spans = []
            for word_start in range(len(sentence_words)):
                for word_end in range(word_start, len(sentence_words)):
                    if sum(sentence_subword_lengths[word_start:word_end]) <= max_mention_length:
                        entity_spans.append(
                            (word_start_char_positions[word_start], word_end_char_positions[word_end])
                        )
                        original_word_spans.append(
                            (word_start, word_end + 1)
                        )

            examples.append(dict(
                text=text,
                words=sentence_words,
                entity_spans=entity_spans,
                original_word_spans=original_word_spans,
            ))

    return examples


def is_punctuation(char):
    cp = ord(char)
    if (cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False

In [ ]:
from datasets import load_dataset

dataset = load_dataset("conll2003")

Downloading:   0%|          | 0.00/2.58k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/983k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
model = LukeForEntitySpanClassification.from_pretrained("studio-ousia/luke-large-finetuned-conll-2003")
model.eval()
model.to("cuda")

# Load the tokenizer
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-large-finetuned-conll-2003")

Some weights of the model checkpoint at studio-ousia/luke-large-finetuned-conll-2003 were not used when initializing LukeForEntitySpanClassification: ['luke.embeddings.position_ids']
- This IS expected if you are initializing LukeForEntitySpanClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LukeForEntitySpanClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
test_documents = load_documents("eng.testb")
test_examples = load_examples(test_documents)

100%|██████████| 231/231 [00:05<00:00, 41.03it/s]


In [ ]:
len(test_documents[1]['sentence_boundaries'])
# test_examples[22]['text']

13

In [ ]:
sentence = 0
tokens = 0
ents = 0

import time

t = 0

s = 0
for i in range(len(test_documents)):
  offset = len(test_documents[i]['sentence_boundaries']) - 1
  raw_test = test_examples[s]['text']
  s += offset

  start = time.time()
  doc = nlp(raw_test)
  t += (time.time() - start)
  tokens += len(doc)
  sentence += len(list(doc.sents))
  ent = [1 if e.label_ == 'PERSON' else 0 for e in doc.ents]
  ents += sum(ent)

print('s', sentence)
print('t', tokens)
print('e', ents)
print('t', t)

s 3085
t 45193
e 1432
t 8.7182776927948


In [ ]:
gt_token = 0
gt_ent = 0

for item in dataset['test']:
  gt_token += len(item['tokens'])
  per = [1 if x == 1 else 0 for x in item['ner_tags']]
  gt_ent += sum(per)

print('t', gt_token)
print('e', gt_ent)

t 46435
e 1617


In [ ]:
with open('/content/nlp-project-dev-data-articles/set1/a1.txt', 'r') as f:
  x = f.read()
  d = nlp(x)
  print(len(d))
  for t in d:
    print(t)

In [7]:
import os

In [25]:
sentence = 0
tokens = 0
# ents = 0

for i in range(1,2):
  file = 'a' + str(i) + '.txt'
  print(file)
  with open(os.path.join('/content/nlp-project-dev-data-articles/set2', file), 'r') as f:
    text = f.read()
    doc = nlp(text)
    tokens += len(doc)
    sentence += len(list(doc.sents))
    # ent = [1 if e.label_ == 'PERSON' else 0 for e in doc.ents]
    # ents += sum(ent)
    with open('o.txt', 'w') as f:
      for e in doc.ents:
        if e.label_ == "PERSON":
          f.write(e.text + ' ' + e.label_ + '\n')

# print('s', sentence)
# print('t', tokens)
# print('e', ents)

a1.txt
